In [33]:
import pandas as pd
from sql import get_data
import warnings
warnings.filterwarnings("ignore")
import psycopg2 # Needed to get database errors when uploading dataframe
import sqlalchemy
import nltk  
import string
from nltk.corpus import stopwords
import os
import psycopg2 # Needed to get database errors when uploading dataframe
import sql
import sqlalchemy

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rebeccastolze/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
clean_jobs = get_data('SELECT * FROM capstone.jobs_translated')

In [3]:
jobs_new = clean_jobs[['id','job_id', 'posted_date', 'company_name', 'job_title_en2', 'job_description', 'city', 'state', 'country']]

In [4]:
jobs_new['job_title_en'] = jobs_new['job_title_en2']

In [5]:
jobs_complete = jobs_new[['id','job_id', 'posted_date', 'company_name', 'job_title_en', 'job_description', 'city', 'state', 'country']]

In [32]:
jobs_complete.head(50)

,job_id,posted_date,company_name,job_title_en,job_description,city,state,country,search_country,world_region
0,2022-04-07_Munich_Embedded Communication Elect...,2022-04-07,Volocopter GmbH,Embedded Communication Electronics - VoloConne...,Your future Volocopter crew:Within a highly mo...,Munich,Bavaria,Germany,Germany,Europe
1,"2022-05-11_Us_Director of Health, Safety & Env...",2022-05-11,Modern Aviation,"Director Health, Safety & Environment",Primary Purpose of the PositionCreates and imp...,Us,None,France,France,Europe
2,2022-05-01_Bouguenais_Ingenieur QSE Alternant H/F,2022-05-01,IRT Jules Verne,Ingenieur QSE Alternant,The Jules Verne Technological Research Institu...,Bouguenais,None,France,France,Europe
3,2022-04-23_Cugnaux_Ingénieur Développement Str...,2022-04-23,AURA-AERO,Structural Development Engineer,"Description of the company ✈AURA AERO, a Frenc...",Cugnaux,None,France,France,Europe
4,2022-05-25_Hamburg_Shell Graduate Programme 20...,2022-05-25,Shell,Shell Graduate Programme 2022 (Commercial) - G...,Are you in the final year of your studies or g...,Hamburg,Hamburg,Germany,Germany,Europe
5,2022-05-16_Aix-en-Provence_Développeur back-end,2022-05-16,Cap4 Lab,Back end developer,"With Cap4 Lab, you join a growing internationa...",Aix-en-Provence,Provence-Alpes-Côte d'Azur,France,France,Europe
6,2022-03-22_Paris_Infrastructure Engineer - (Fl...,2022-03-22,SITA,Infrastructure Engineer - (Flex/Hybrid Options),As owner of one of the largest secure private ...,Paris,Île-de-France,France,France,Europe
7,2022-05-16_Greater Marseille Metropolitan Area...,2022-05-16,FERCHAU Aviation Group - France,Project Manager,Job description: Flight simulation project man...,Marseille,Provence-Alpes-Côte d''Azur,France,France,Europe
8,2022-05-18_Seine-et-Marne_Ingénieur Assurance ...,2022-05-18,Modis,Aerospace Supplier Quality Assurance Engineer,Your mission Modis Tech Talent Services* is th...,Seine-et-Marne,None,France,France,Europe
9,2022-04-28_Hasparren_Stage « Assistant chef de...,2022-04-28,LAUAK GROUPE,“ERP coordination project manager assistant” i...,"CompanyA fast-developing family group, LAUAK i...",Hasparren,None,France,France,Europe


In [31]:
jobs_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1779 entries, 0 to 1778
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   job_id           1779 non-null   object        
 1   posted_date      1779 non-null   datetime64[ns]
 2   company_name     1779 non-null   object        
 3   job_title_en     1779 non-null   object        
 4   job_description  1779 non-null   object        
 5   city             1779 non-null   object        
 6   state            1118 non-null   object        
 7   country          1779 non-null   object        
 8   search_country   1779 non-null   object        
 9   world_region     1165 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 139.1+ KB


In [10]:
region_category = {
    'Europe' : ['Ukraine', 'France', 'Spain', 'Sweden', 'Norway', 'Germany', 'Finland', 'Poland', 'Italy', 'United Kingdom', 
                'Romania', 'Belarus', 'Kazakhstan', 'Greece', 'Bulgaria', 'Iceland', 'Hungary', 'Portugal', 'Serbia', 'Austria', 
                'Czech Republic', 'Ireland', 'Lithuania', 'Latvia', 'Croatia', 'Bosnia and Herzegovina', 'Slovakia', 'Estonia', 
                'Denmark', 'Switzerland', 'Netherlands', 'Moldova', 'Belgium', 'Albania', 'North Macedonia', 'Slovenia', 
                'Montenegro', 'Kosovo', 'Azerbaijan', 'Luxembourg', 'Andorra', 'Malta', 'Liechtenstein', 'San Marino', 
                'Monaco', 'Vatican City', 'Cyprus',  'Czechia', 'Czech Republic'],

    'Africa' : ['Nigeria', 'Ethiopia', 'Egypt', 'DR Congo', 'Tanzania', 'South Africa', 'Kenya', 'Uganda', 'Algeria', 
                'Sudan', 'Morocco', 'Angola', 'Mozambique', 'Ghana', 'Madagascar', 'Cameroon', 'Niger', 'Burkina Faso', 'Mali', 
                'Malawi', 'Zambia', 'Senegal', 'Chad', 'Somalia', 'Zimbabwe', 'Guinea', 'Rwanda', 'Benin', 'Burundi', 'Tunisia', 
                'South Sudan', 'Togo', 'Sierra Leone', 'Libya', 'Congo', 'Liberia', 'Central African Republic', 'Mauritania', 'Eritrea', 
                'Namibia', 'Gambia', 'Botswana', 'Gabon', 'Lesotho', 'Guinea-Bissau', 'Equatorial Guinea', 'Mauritius', 'Eswatini', 'Djibouti'],

'Middle East' : ['Turkey', 'Bahrain', 'Kuwait', 'Oman', 'Qatar', 'Saudi Arabia', 'United Arab Emirates', 'Yemen', 'Abkhazia', 'Armenia', 
                'Artsakh', 'Azerbaijan', 'Georgia', 'South Ossetia', 'Iraq', 'Israel', 'Jordan', 'Lebanon', 'Palestine', 'Syria', 'Iran', 
                'Akrotiri and Dhekelia', 'Cyprus'],
                
'North America' : ['USA', 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 
                'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 
                'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
                'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 
                'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 
                'West Virginia', 'Wisconsin', 'Wyoming', 'Canada', 'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 
                'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 
                'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'],

'Central America' : ['Mexico', 'Guatemala', 'Cuba', 'Haiti', 'Dominican Republic', 'Honduras', 'Nicaragua', 'El Salvador', 'Costa Rica', 'Panama', 
                    'Jamaica', 'Trinidad and Tobago', 'Belize', 'Bahamas', 'Barbados', 'Saint Lucia', 'Grenada', 'Antigua and Barbuda', 'Dominica', 
                    'Saint Kitts and Nevis'],

'South America' : ['Argentina', 'Bolivia', 'Bouvet Island', 'Brazil', 'Chile', 'Colombia', 'Ecuador', 'Falkland Islands', 'French Guiana', 
                'Guyana', 'Paraguay', 'Peru', 'South Georgia', 'Suriname', 'Uruguay', 'Venezuela', 'South Sandwich Islands'],

'South Asia' : ['Afghanistan', 'Bangladesh', 'Bhutan', 'India', 'Nepal', 'Maldives', 'Pakistan', 'Sri Lanka'],

'Southeast Asia' : ['Brunei', 'Cambodia', 'East Timor', 'Indonesia', 'Laos', 'Malaysia', 'Myanmar', 'Philippines', 'Singapore', 'Thailand', 'Vietnam'],

'Central Asia' : ['Kazakhstan', 'Kyrgyzstan', 'Tajikistan', 'Turkmenistan', 'Uzbekistan'],

'East Asia' : ['China', 'Hong Kong', 'Macau', 'Japan', 'Mongolia', 'North Korea', 'South Korea', 'Taiwan']

}

In [28]:
jobs_complete = jobs_complete.drop(columns='job_title_trans', axis=1)

In [7]:
jobs_complete['search_country'] = jobs_complete['country']

In [19]:
world_regions = pd.Series([])

for row in range(len(jobs_complete)):

    for key in region_category:

        for value in region_category[key]:
            
            if jobs_complete['country'][row] == value:
                world_regions[row] = key
   
jobs_complete.insert(9, "world_region", world_regions)

In [21]:
jobs_complete['job_title_trans'] = jobs_complete['job_title_en']

In [25]:
new_stopwords = ['(m/f/d)', 'H/F', 'M/F', '(m/f)', 'F/M', '(d/f/m)', '(m/w/d)', '(m/f/diverse)', 'F/H', 
            '(F/M)', '( F/M)', 'm/f', '(m/f/x)', '(M/F/D)', '(all genders)', '(M/F)', '(d/m/f)', '(m / w / d)', 
            '(D/W/M)', '(f/m/d)', '(f/m/x)', '(f/m/diverse)', '(F / H)', '(He/She/They)', '(She/He/They)', 
            'F / H', '(H/F)', '(w/m/d)', '(3rd Shift)', '(1st Shift)','(On-Site', '( F/M)', '(Direct Hire)', 'REMOTE', 'Remote', 'remote']
#nltk.download('stopwords')     #### I don't know if it has to be in the function       

#global stopwords
stopwords = stopwords.words('english')
stopwords.extend(new_stopwords)

clean_job_title = pd.Series([])

for row in range(len(jobs_complete)):
    clean_job_title[row] = ' '.join([word for word in jobs_complete.loc[row, 'job_title_trans'].split() if word not in stopwords])

jobs_complete.insert(3, "job_title_en", clean_job_title)

In [30]:
jobs_complete['posted_date'] = pd.to_datetime(jobs_complete['posted_date'], format="%Y-%m-%d")

In [34]:
schema = os.getenv('schema')
from sql import engine
for row in range(len(jobs_complete)):
    try:
        jobs_complete.iloc[row:row+1, 0:10].to_sql(name='jobs',                 # Name of DF table
                                                con=engine,                     # Engine or connection
                                                if_exists='append',             # Just add new values in existing table
                                                schema=schema,                  # Use schema that was defined earlier in .sql
                                                index=False,                    # Write DataFrame index as a column
                                                chunksize=1,                    # Specify the number of rows in each batch to be written at a time
                                                method='multi')                 # Pass multiple values in a single INSERT clause
        print(f"row {row} of jobs was inserted successfully.")
    except:
        print(f"row {row} of jobs was ignored.")
        continue

row 0 of jobs was inserted successfully.
row 1 of jobs was inserted successfully.
row 2 of jobs was inserted successfully.
row 3 of jobs was inserted successfully.
row 4 of jobs was inserted successfully.
row 5 of jobs was inserted successfully.
row 6 of jobs was inserted successfully.
row 7 of jobs was inserted successfully.
row 8 of jobs was inserted successfully.
row 9 of jobs was inserted successfully.
row 10 of jobs was inserted successfully.
row 11 of jobs was inserted successfully.
row 12 of jobs was inserted successfully.
row 13 of jobs was inserted successfully.
row 14 of jobs was inserted successfully.
row 15 of jobs was inserted successfully.
row 16 of jobs was inserted successfully.
row 17 of jobs was inserted successfully.
row 18 of jobs was inserted successfully.
row 19 of jobs was inserted successfully.
row 20 of jobs was inserted successfully.
row 21 of jobs was inserted successfully.
row 22 of jobs was inserted successfully.
row 23 of jobs was inserted successfully.
ro